In [14]:
import pandas as pd

# Step 1: Load the dataset
file_path = r"C:\Users\HP\OneDrive\Documents\netflix_dataset.xlsx" # Adjust path if needed
df = pd.read_excel(file_path)

# Step 2: Rename columns (clean and uniform)
df.columns = df.columns.str.strip().str.lower().str.replace(r'[^a-z0-9]+', '_', regex=True)

# Step 3: Remove duplicate rows
df.drop_duplicates(inplace=True)

# Step 4: Handle missing values
df.dropna(subset=['date_added'], inplace=True)  # Drop rows where date is missing

# Fill NA in selected columns with 'Unknown'
fill_cols = ['director', 'cast', 'country', 'rating']
for col in fill_cols:
    df[col] = df[col].fillna('Unknown')

# Step 5: Convert 'date_added' to datetime and extract parts
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df.dropna(subset=['date_added'], inplace=True)  # Drop if still unparseable
df['added_year'] = df['date_added'].dt.year
df['added_month'] = df['date_added'].dt.month
df['added_day'] = df['date_added'].dt.day

# Step 6: Handle 'duration' column
if 'duration' in df.columns:
    df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)', expand=True)
    df['duration_value'] = pd.to_numeric(df['duration_value'], errors='coerce').fillna(0).astype(int)
    df['duration_unit'] = df['duration_unit'].fillna('Unknown')
    df.drop('duration', axis=1, inplace=True)

# Step 7: Standardize text fields
text_cols = ['type', 'rating', 'country']
for col in text_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

# Clean multi-country entries
df['country'] = df['country'].apply(
    lambda x: ', '.join(sorted([s.strip().title() for s in x.split(',')])) if isinstance(x, str) else x
)

# Step 8: Fix numeric types
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').fillna(-1).astype(int)

# Step 9: Save cleaned dataset (Optional)
df.to_csv('netflix_titles_cleaned.csv', index=False)
print("Cleaned dataset saved as 'netflix_titles_cleaned.csv'")

# Preview cleaned data
print("\n Sample cleaned data:")
print(df.head())

Cleaned dataset saved as 'netflix_titles_cleaned.csv'

 Sample cleaned data:
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  Tv Show          Blood & Water          Unknown   
2      s3  Tv Show              Ganglands  Julien Leclercq   
3      s4  Tv Show  Jailbirds New Orleans          Unknown   
4      s5  Tv Show           Kota Factory          Unknown   

                                                cast        country  \
0                                            Unknown  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...        Unknown   
3                                            Unknown        Unknown   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

  date_added  release_year rating  \
0 2021-09-25          2020  Pg-13   
1 2021-09-24          2021  Tv-Ma   
2 2021-09-24